In [1]:
# # read unlabelled dataset
# with open("unlabelled_dataset.txt", encoding='utf-8') as f:
#     text = f.read()

# print("Content Length: ", len(text))
# print("Top 1000: ", text[:1000])

In [2]:
import pandas as pd
df = pd.read_csv("newspeak.csv")
text = "".join([v for v in df["Newspeak Translation"].unique()])

In [3]:
# collect all unique characters from text:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("vocab_size: ", vocab_size)
print("Characters: ", "".join(chars))

vocab_size:  40
Characters:   '.BCDGHINOPRSTUWabcdefghijklmnoprstuvwy


In [4]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode= lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[24, 25, 0, 35, 24, 21, 33, 21]
hi there


In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([6800])


In [6]:
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
context_length = 8
train_data[:context_length+1]

tensor([15, 30, 26, 31, 39,  0, 25, 34,  0])

In [8]:
x = train_data[:context_length]
y = train_data[1:context_length+1]

In [9]:
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - context_length, (batch_size, ))
    x = torch.stack([data[i:i + context_length] for i in ix])
    y = torch.stack([data[i + 1:i + context_length + 1] for i in ix])
    return x, y

xb, yb = get_batch("train")

In [10]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # Batch, Time, Channel
        if targets is None:
            loss = None
        else:
            # reshape B, T, C
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


model = BigramLM(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss.shape)
print(loss)

torch.Size([32, 40])
torch.Size([])
tensor(4.4530, grad_fn=<NllLossBackward0>)


In [11]:
print(decode(model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

 gStrCuapDI.mBaaUmGylOwrm. cU.rW'TivlasNdHnGypwhwmbscgwcgeGHhtbBlewtHvnRu.HvsTTtiBfHlS SeBwooPCNCtsCN


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

/Users/cuburtbalanon/anaconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

1.8287968635559082


In [19]:
print(decode(model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

 thie th dode gy.The mplus t un'the pl.We crWe arusth.Therun'tooodey uno the.Wey.I beriey cerodot unp


In [ ]:
torch.manual_seed(1337)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
w = q @ k.transpose(-2, -1)


